In [2]:
!nvidia-smi

Tue Jun  9 22:53:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:04:00.0 Off |                  N/A |
| 44%   65C    P2   279W / 280W |  20925MiB / 24220MiB |     85%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:05:00.0 Off |                  N/A |
| 91%   88C    P2   275W / 280W |  15839MiB / 24220MiB |     85%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [64]:
######### IMPORTING NECESSARY MODULES #########
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torchvision import transforms, datasets, models
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm


In [ ]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)

**Dataloading Scheme**

In [ ]:
trainlist = 'new_classification_lists/new_classification_trainlist.txt'
validlist ='new_classification_lists/new_classification_vallist.txt'
# testlist = 'new_classification_lists/new_classification_testlist.txt'

In [ ]:
# Create a mapping of Unique product Ids to Labels(0 to 31127 classes)
# output = dictionary containing mapping of each upc to a label from (0 to 31127)  

with open(trainlist, mode = 'r') as f:
    
    Y=[]
    for line in f:
        path, UPC = line[:-1].split(',')

        Y.append(UPC)
        
upc_list = sorted(set(Y))

upc_dict = { upc_list[i] :i for i in range(0, len(upc_list) ) }

In [ ]:
class mydataset():    

    def __init__(self, classification_list, upc_dict, name):

        super(mydataset).__init__()
        
        self.X = []
        self.Y = []
        
        with open(classification_list, mode = 'r') as f:
            
            for line in f:
                path, UPC = line[:-1].split(',')

                self.X.append(path)
                self.Y.append(upc_dict[UPC])
        

        if name == 'valid':
            self.transform = transforms.Compose([   transforms.RandomResizedCrop(224),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                    std=[0.229, 0.224, 0.225])
                                                ])
        else:
            self.transform = transforms.Compose([   transforms.RandomResizedCrop(224),
                                                transforms.RandomHorizontalFlip(),
                                                transforms.RandomErasing(p=0.5)
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                    std=[0.229, 0.224, 0.225])
                                                                                            ])
    
    def __getitem__(self,index):
        
        image = self.X[index]        
        label = float(self.Y[index])
        
        image = (Image.open(image))
               
        image = self.transform(image)
        
        return image, torch.as_tensor(label).long()
        
  
    def __len__(self):
        return len(self.X)

In [ ]:
# #### Train Dataloader #### 
train_dataset = mydataset(trainlist, upc_dict, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 256, num_workers=16,pin_memory=True)


#### Validation Dataloader #### 
validation_dataset = mydataset(validlist, upc_dict, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 256, num_workers=16,pin_memory=True)

**RESNET Architecture**

**Model Definition**

In [ ]:
model = models.resnet50(pretrained=False, num_classes=31128)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4, momentum=0.9)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 30, gamma = 0.1)

checkpoint = torch.load('/home/ironman/sourabh/saved_models/resnet50_baseline/model_89_resnet50.pth')
model.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])
lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
    
model = nn.DataParallel(model,device_ids=[1,2]).to(device)
model

**Training Method**

In [ ]:
def train(model, data_loader, test_loader, epochs, task='Classification'):
    model.train()

    for epoch in range(epochs):
        avg_loss = 0.0
        
        scheduler.step()
    
        for batch_num, (feats, labels) in enumerate(data_loader):
            feats, labels = feats.to(device), labels.to(device)
            
            
            outputs = model(feats)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            
            loss.backward()
            optimizer.step()
            
            avg_loss += loss.item()

#             if batch_num % 200 == 199:
#                 print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+38, batch_num+1, avg_loss/200))
#                 avg_loss = 0.0    

            print('loss', avg_loss)

            torch.cuda.empty_cache()
            del feats
            del labels
            del loss
        
        print('Epoch: ', epoch+1)

        print('training loss = ', avg_loss/len(data_loader))
        train_loss.append(avg_loss/len(data_loader))

        ## Check performance on validation set after an Epoch
        valid_loss, valid_acc = test_classify(model, test_loader)
        print('Val Loss: {:.4f}\tVal Accuracy: {:.4f}'.format(valid_loss, valid_acc))


        
        
        #########save model checkpoint #########
        torch.save({
#             'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
#             'lr_scheduler': lr_scheduler.state_dict()
            }, 'saved_model_checkpoints/mymodel')


def test_classify(model, test_loader):
    model.eval()
    test_loss = []
    accuracy = 0
    total = 0

    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats)
        
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels = pred_labels.view(-1)
        
        loss = criterion(outputs, labels.long())
        
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
        test_loss.extend([loss.item()]*feats.size()[0])
        del feats
        del labels

    model.train()
    return np.mean(test_loss), accuracy/total



**Hyperparameters**

In [ ]:
# Loss Function
criterion = nn.CrossEntropyLoss()

# Optimizer
# optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4, momentum=0.9)

# Epochs
num_Epochs = 1

In [ ]:
train_loss= []
v_loss = []
v_acc = []

In [ ]:
train(model, train_dataloader, validation_dataloader, epochs = 1) 

In [66]:
######### Load saved model from checkpoint  #########

checkpoint = torch.load('saved_model_checkpoints/CosineLR')
# epoch = checkpoint['epoch']
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
# training_loss = checkpoint['loss']


In [69]:
checkpoint['model_state_dict']

OrderedDict([('module.conv1.weight',
              tensor([[[[-9.5057e-02, -5.8204e-02, -2.9081e-02,  ..., -1.2586e-02,
                          2.4411e-02,  3.3882e-02],
                        [-6.4341e-02, -2.0726e-02,  4.1053e-03,  ...,  3.1044e-02,
                          7.5721e-02,  7.1771e-02],
                        [-3.5912e-02,  1.4931e-02,  3.8293e-02,  ...,  7.3437e-02,
                          1.1193e-01,  8.6723e-02],
                        ...,
                        [-9.1109e-04,  4.0724e-02,  6.9906e-02,  ...,  1.3028e-01,
                          1.6107e-01,  1.2826e-01],
                        [ 6.3528e-02,  1.0409e-01,  1.2941e-01,  ...,  1.8554e-01,
                          2.1313e-01,  1.9661e-01],
                        [ 9.2565e-02,  1.1889e-01,  1.3628e-01,  ...,  1.8157e-01,
                          2.1713e-01,  2.2238e-01]],
              
                       [[ 1.4029e-01,  6.3496e-02,  7.7932e-02,  ..., -2.8598e-02,
                         

In [ ]:
# plt.figure(figsize=(8,8))
# x = np.arange(91,121)
# plt.plot(x, train_loss)
# plt.xlabel('Epochs', fontsize =16)
# plt.ylabel('Training Loss', fontsize =16)
# plt.title('Training Loss v/s Epochs',fontsize =16)


plt.figure(figsize=(8,8))
x = np.arange(91,121)
plt.plot(x, train_loss, label = 'Training Loss')
plt.plot(x, v_loss, label = 'Validation Loss')
plt.xlabel('Epochs', fontsize =16)
plt.ylabel('Loss', fontsize =16)
plt.title('Loss v/s Epochs',fontsize =16)
plt.legend(fontsize=16)

In [ ]:
y = x